In [3]:
%load_ext autoreload
%autoreload 2

import pickle
import pandas as pd
import os
import openai
import numpy as np
import ipdb
import re
from tqdm import tqdm

from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
import spacy
import scipy

nlp = spacy.load("en_core_web_sm")
openai.api_key= os.environ['OPENAI_KEY']

from data_utils import *
from gpt3_utils import *
from eval_utils import *

pd.set_option('max_rows',500,'max_colwidth',10000)
pd.options.display.float_format = "{:,.2f}".format

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#Loading BC5CDR Chemical and Disease Training Sets

chemical_train = pd.read_csv('../data/bc5cdr_chemical.train.processed.tsv',sep='\t')
disease_train = pd.read_csv('../data/bc5cdr_disease.train.processed.tsv',sep='\t')

In [5]:
disease_train['sent_len'] = [len(s) for s in disease_train.sents]

In [6]:
train_half1 = disease_train[0:int(len(disease_train)/2)]
train_half2 = disease_train[int(len(disease_train)/2):]

In [7]:
def create_prompt_dataset(train_df, test_df, seed, few_shot_num, number_of_test_samples, selection_strategy,sep=', ',train_df2=None):
    
    random = np.random.RandomState(seed)
    
    few_shot_prompt, chosen_prompt_ids = create_few_shot_prompt(train_df, random, few_shot_num, selection_strategy)
    
    test_df['test_ready_prompt'] = [few_shot_prompt+'\n\n'+empty_prompt for empty_prompt in test_df['empty_prompts']]
    
    if number_of_test_samples != 'all':
        #Making sure the samples are the same as the first batch
        random = np.random.RandomState(42)
        random.permutation(train_df2.index)
        chosen_test_ids = random.permutation(test_df.index)[:number_of_test_samples]
        
        chosen_test_df = test_df.loc[chosen_test_ids]
    else:
        chosen_test_df = test_df
    
    return {'seed':seed,'few_shot_prompt': few_shot_prompt, 'chosen_prompt_ids':chosen_prompt_ids,'sep': sep,'test_df': chosen_test_df}

In [8]:
def test_prompt_selection(engine, train_df, eval_df, few_shot_seeds, dev_set_seeds, few_shot_size=5, eval_size=50, sampling_strategy='random'):

    result_by_seeds = []
    result_dfs_by_seeds = {}

    for few_shot_seed in few_shot_seeds:
        
        for dev_set_seed in dev_set_seeds:
            
            if dev_set_seed > 0:
                permuted_eval_df = eval_df.sample(frac=1, random_state=np.random.RandomState(dev_set_seed))
            else:
                permuted_eval_df = eval_df
            
            dev_data = create_prompt_dataset(train_df, permuted_eval_df, few_shot_seed, few_shot_size, eval_size, sampling_strategy,train_df2=train_half1)
                
            dev_df = dev_data['test_df']
            prompts = dev_df.test_ready_prompt.values

            result_df = run_gpt3_on_df(engine, dev_df, prompts, max_tokens=30, sep=dev_data['sep'], logit_bias=10, sep_logit_bias=10, new_line_logit_bias=10)

            df = create_bio_preds(result_df, "predictions")
            f1, precision, recall = conlleval_eval(df.ner_seq,df.bio_preds)

            result_by_seeds.append((few_shot_seed, dev_set_seed, f1, precision, recall, prompts[0]))
            result_dfs_by_seeds[(few_shot_seed, dev_set_seed)] = result_df
        
    return pd.DataFrame(result_by_seeds, columns=["few_shot_seed", "dev_set_seed", "f1", "precision", "recall", "prompt"]), result_dfs_by_seeds

def create_few_shot_prompt(train_df, random, few_shot_num, selection_strategy):
    
    if selection_strategy == 'random':
        chosen_inds = random.permutation(train_df.index)[:few_shot_num]
    elif selection_strategy == 'stratified':
        chosen_inds = []
        sorted_available_num_entites = np.sort(train_df.num_entities.unique())
        
        if len(sorted_available_num_entites) > few_shot_num:
            for i in range(few_shot_num):
                chosen_inds.append(random.permutation(train_df[train_df.num_entities == sorted_available_num_entites[i]].index)[0])
        else:
            #TODO: Finish logic when the number of diverse entity numbers is lower than the number of few shot examples 
            for num_ents in sorted_available_num_entites:
                chosen_inds.append(random.permutation(train_df[train_df.num_entities == num_ents].index)[0])
                        
    few_shot_prompt_list = train_df.loc[chosen_inds, 'prompts'].values
    few_shot_prompt = '\n\n'.join(few_shot_prompt_list)
    
    return few_shot_prompt, chosen_inds

In [9]:
many_ents_train_half1 = train_half1[train_half1['num_entities'] > 3]

In [10]:
many_ents_train_half1

,Unnamed: 0,sents,ner_seq,entities,num_entities,num_tokens,orig_tok_sent,prompts,empty_prompts,sent_len
55,55,"Eleven of the cocaine abusers and none of the controls had ECG evidence of significant myocardial injury defined as myocardial infarction, ischemia, and bundle branch block.",O O O O O O O O O O O O O O O B I O O B I O B O O B I I O,"['myocardial injury', 'myocardial infarction', 'ischemia', 'bundle branch block']",4,20,"Eleven of the cocaine abusers and none of the controls had ECG evidence of significant myocardial injury defined as myocardial infarction , ischemia , and bundle branch block .","Sentence: Eleven of the cocaine abusers and none of the controls had ECG evidence of significant myocardial injury defined as myocardial infarction, ischemia, and bundle branch block.\nDiseases: myocardial injury, myocardial infarction, ischemia, bundle branch block","Sentence: Eleven of the cocaine abusers and none of the controls had ECG evidence of significant myocardial injury defined as myocardial infarction, ischemia, and bundle branch block.\nDiseases:",173
65,65,"Visual toxicity was of retinal origin and was characterized by a tritan-type dyschromatopsy, sometimes associated with a loss of visual acuity and pigmentary retinal deposits.",B I O O O O O O O O O O O O B O O O O B I I I I O B I I O,"['visual toxicity', 'dyschromatopsy', 'a loss of visual acuity', 'pigmentary retinal deposits']",4,21,"Visual toxicity was of retinal origin and was characterized by a tritan - type dyschromatopsy , sometimes associated with a loss of visual acuity and pigmentary retinal deposits .","Sentence: Visual toxicity was of retinal origin and was characterized by a tritan-type dyschromatopsy, sometimes associated with a loss of visual acuity and pigmentary retinal deposits.\nDiseases: visual toxicity, dyschromatopsy, a loss of visual acuity, pigmentary retinal deposits","Sentence: Visual toxicity was of retinal origin and was characterized by a tritan-type dyschromatopsy, sometimes associated with a loss of visual acuity and pigmentary retinal deposits.\nDiseases:",175
105,105,Induction of intravascular coagulation and inhibition of fibrinolysis by injection of thrombin and tranexamic acid (AMCA) in the rat gives rise to pulmonary and renal insufficiency resembling that occurring after trauma or sepsis in man.,O O B I O O O O O O O O O O O O O O O O O O O O B I I I O O O O B O B O O O,"['intravascular coagulation', 'pulmonary and renal insufficiency', 'trauma', 'sepsis']",4,21,Induction of intravascular coagulation and inhibition of fibrinolysis by injection of thrombin and tranexamic acid ( AMCA ) in the rat gives rise to pulmonary and renal insufficiency resembling that occurring after trauma or sepsis in man .,"Sentence: Induction of intravascular coagulation and inhibition of fibrinolysis by injection of thrombin and tranexamic acid (AMCA) in the rat gives rise to pulmonary and renal insufficiency resembling that occurring after trauma or sepsis in man.\nDiseases: intravascular coagulation, pulmonary and renal insufficiency, trauma, sepsis",Sentence: Induction of intravascular coagulation and inhibition of fibrinolysis by injection of thrombin and tranexamic acid (AMCA) in the rat gives rise to pulmonary and renal insufficiency resembling that occurring after trauma or sepsis in man.\nDiseases:,237
149,149,"The relationship of arthritis and sexual dysfunction was investigated among 169 patients with rheumatoid arthritis, osteoarthritis and spondyloarthropathy, 130 of whom were pair-matched to controls.",O O O B O B I O O O O O O B I O B O B O O O O O O O O O O O,"['arthritis', 'sexual dysfunction', 'rheumatoid arthritis', 'osteoarthritis', 'spondyloarthropathy']",5,23,"The relationship of arthritis and sexual dysfunction was investigated among 169 patients with rheumatoid arthritis , osteoarthritis and spondyloarthropathy , 130 of whom were pair - matched to controls .","Sentence: The relationship of arthritis and sexual dysfunctio

In [96]:
disease_performance_by_prompt_sel_many_ents, disease_results_by_prompt_sel_many_ents = test_prompt_selection('davinci',
                                                                                         many_ents_train_half1, 
                                                                                         train_half2,
                                                                                         [0,2],
                                                                                         [0])

<ipython-input-91-144fed795d56>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['test_ready_prompt'] = [few_shot_prompt+'\n\n'+empty_prompt for empty_prompt in test_df['empty_prompts']]
50it [00:31,  1.57it/s]
0it [00:00, ?it/s]

processed 1125 tokens with 37 phrases; found: 106 phrases; correct: 26.
accuracy:  82.84%; (non-O)
accuracy:  82.84%; precision:  24.53%; recall:  70.27%; FB1:  36.36%
                X: precision:  24.53%; recall:  70.27%; FB1:  36.36%  106


50it [00:34,  1.44it/s]

processed 1125 tokens with 37 phrases; found: 134 phrases; correct: 26.
accuracy:  82.49%; (non-O)
accuracy:  82.49%; precision:  19.40%; recall:  70.27%; FB1:  30.41%
                X: precision:  19.40%; recall:  70.27%; FB1:  30.41%  134


In [101]:
disease_performance_by_prompt_sel_many_ents.sort_values('f1',ascending=False)[["few_shot_seed", "dev_set_seed", "f1", "precision", "recall"]]

,few_shot_seed,dev_set_seed,f1,precision,recall
0,0,0,36.36,24.53,70.27
1,2,0,30.41,19.40,70.27


In [106]:
disease_performance_by_prompt_sel_many_ents, disease_results_by_prompt_sel_many_ents = test_prompt_selection('ada',
                                                                                         many_ents_train_half1, 
                                                                                         train_half2,
                                                                                         [0,2,4,6],
                                                                                         [0,1],sampling_strategy='stratified')

<ipython-input-91-144fed795d56>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['test_ready_prompt'] = [few_shot_prompt+'\n\n'+empty_prompt for empty_prompt in test_df['empty_prompts']]
50it [00:11,  4.42it/s]
<ipython-input-91-144fed795d56>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['test_ready_prompt'] = [few_shot_prompt+'\n\n'+empty_prompt for empty_prompt in test_df['empty_prompts']]
1it [00:00,  7.84it/s]

processed 1125 tokens with 37 phrases; found: 100 phrases; correct: 26.
accuracy:  88.27%; (non-O)
accuracy:  88.27%; precision:  26.00%; recall:  70.27%; FB1:  37.96%
                X: precision:  26.00%; recall:  70.27%; FB1:  37.96%  100


50it [00:08,  5.78it/s]

processed 1125 tokens with 37 phrases; found: 75 phrases; correct: 20.
accuracy:  90.76%; (non-O)
accuracy:  90.76%; precision:  26.67%; recall:  54.05%; FB1:  35.71%
                X: precision:  26.67%; recall:  54.05%; FB1:  35.71%  75


In [98]:
print(disease_results_by_prompt_sel_many_ents[0,0].test_ready_prompt.values[0])

Sentence: Autism is a neurodevelopmental disorder presenting before 3 years of age with deficits in communication and social skills and repetitive behaviors .
Diseases: neurodevelopmental disorder, autism, deficits in communication and social skills, repetitive behaviors

Sentence: He developed acute neurologic symptoms of mental confusion , disorientation and irritability , and then lapsed into a deep coma , lasting for approximately 40 hours during the first dose ( day 2 ) of 5 - fluorouracil and folinic acid infusion .
Diseases: disorientation, confusion, irritability, coma

Sentence: After most infections causing erythema multiforme and myocarditis were ruled out , a drug - induced allergic reaction was suspected .
Diseases: erythema multiforme, infections, drug - induced allergic reaction, myocarditis

Sentence: Compared with placebo subjects , alprazolam patients developed more adverse reactions ( 21 % v . 0 % ) of depression , enuresis , disinhibition and aggression ; and more s

In [104]:
disease_results_by_prompt_sel_many_ents[0,0]

,Unnamed: 0,sents,ner_seq,entities,num_entities,num_tokens,prompts,empty_prompts,test_ready_prompt,predictions,gpt3_output_predictions,bio_preds,predictions.post
2697,2697,The site of common side effects of sumatriptan .,O O O O O O O O O,[],0,0,Sentence: The site of common side effects of sumatriptan .\nDiseases:,Sentence: The site of common side effects of sumatriptan .\nDiseases:,"Sentence: Autism is a neurodevelopmental disorder presenting before 3 years of age with deficits in communication and social skills and repetitive behaviors .\nDiseases: neurodevelopmental disorder, autism, deficits in communication and social skills, repetitive behaviors\n\nSentence: He developed acute neurologic symptoms of mental confusion , disorientation and irritability , and then lapsed into a deep coma , lasting for approximately 40 hours during the first dose ( day 2 ) of 5 - fluorouracil and folinic acid infusion .\nDiseases: disorientation, confusion, irritability, coma\n\nSentence: After most infections causing erythema multiforme and myocarditis were ruled out , a drug - induced allergic reaction was suspected .\nDiseases: erythema multiforme, infections, drug - induced allergic reaction, myocarditis\n\nSentence: Compared with placebo subjects , alprazolam patients developed more adverse reactions ( 21 % v . 0 % ) of depression , enuresis , disinhibition and aggression ; and more side - effects , particularly sedation , irritability , impaired memory , weight loss and ataxia .\nDiseases: impaired memory, depression, enuresis, weight loss, irritability, ataxia, aggression\n\nSentence: Doxorubicin treatment resulted in heavy proteinuria ( > 100 mg protein / mg crea ) in 15 / 44 of sgk1 ( + / + ) and 15 / 44 of sgk1 ( - / - ) mice leading to severe nephrotic syndrome with ascites , lipidemia , and hypoalbuminemia in both genotypes .\nDiseases: lipidemia, proteinuria, hypoalbuminemia, nephrotic syndrome, ascites\n\nSentence: The site of common side effects of sumatriptan .\nDiseases:",[common side effects of sumatriptan],"{'id': 'cmpl-4GwmQyAhxiLROcjhXaph3Vamnc6AA', 'object': 'text_completion', 'created': 1639944546, 'model': 'davinci:2020-05-03', 'choices': [{'text': ' common side effects of sumatriptan', 'index': 0, 'logprobs': { ""text_offset"": [ 1603, 1610, 1615, 1623, 1626, 1630, 1632, 1636, 1638, 1638, 1638, 1638, 1638, 1638, 1638, 1638 ], ""token_logprobs"": [ -0.643381, -0.03209907, -0.004247826, -0.6642729, -0.0013111025, -5.0778563e-06, -4.1232483e-06, -2.8682118e-05, -0.10832213, -0.0012306508, -0.0057306094, -1.496964, -0.14029692, -0.47446486, -0.0038809602, -0.13734025 ], ""tokens"": [ "" common"", "" side"", "" effects"", "" of"", "" sum"", ""at"", ""ript"", ""an"", ""\n"", ""\n"", ""The"", "" sum"", "" of"", "" side"", "" effects"", "" of"" ], ""top_logprobs"": [ { "" common"": -0.643381 }, { "" side"": -0.03209907 }, { "" effects"": -0.004247826 }, { "" of"": -0.6642729 }, { "" sum"": -0.0013111025 }, { ""at"": -5.0778563e-06 }, { ""ript"": -4.1232483e-06 }, { ""an"": -2.8682118e-05 }, { ""\n"": -0.10832213 }, { ""\n"": -0.0012306508 }, { ""The"": -0.0057306094 }, { "" sum"": -1.496964 }, { "" of"": -0.14029692 }, { "" side"": -0.47446486 }, { "" effects"": -0.0038809602 }, { "" of"": -0.13734025 } ] }, 'finish_reason': 'stop'}]}",O O O B I I I I O,[common side effects of sumatriptan]
3709,3709,Hemodynamic parameters and lead II electrocardiograph were monitored and recorded continuously .,O O O O O O O O O O O O,[],0,0,Sentence: Hemodynamic parameters and lead II electrocardiograph were monitored and recorded continuously .\nDiseases:,Sentence: Hemodynamic parameters and lead II electrocardiograph were monitored and recorded continuously .\nDiseases:,"Sentence: Autism is a neurodevelopmental disorder presenting before 3 years of age with deficits in communication and social skills and repetitive behaviors .\nDiseases: neurodevelopmental disorder, autism, deficits in communication and social skills, repetitive behaviors\

In [15]:
few_shot_seeds = [42,4,31,9]
disease_performance_by_prompt_sel_stratified, disease_results_by_prompt_sel_stratified = test_prompt_selection('davinci',
                                                                                         train_half1,
                                                                                         train_half2,
                                                                                         few_shot_seeds,
                                                                                         [0],sampling_strategy='stratified')

<ipython-input-7-05736d5e33a3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['test_ready_prompt'] = [few_shot_prompt+'\n\n'+empty_prompt for empty_prompt in test_df['empty_prompts']]
50it [00:32,  1.52it/s]

processed 1125 tokens with 37 phrases; found: 94 phrases; correct: 26.
accuracy:  87.82%; (non-O)
accuracy:  87.82%; precision:  27.66%; recall:  70.27%; FB1:  39.69%
                X: precision:  27.66%; recall:  70.27%; FB1:  39.69%  94


,F1,Precision,Recall
0,39.69,27.66,70.27


<ipython-input-7-05736d5e33a3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['test_ready_prompt'] = [few_shot_prompt+'\n\n'+empty_prompt for empty_prompt in test_df['empty_prompts']]
50it [00:23,  2.08it/s]

processed 1125 tokens with 37 phrases; found: 61 phrases; correct: 27.
accuracy:  94.40%; (non-O)
accuracy:  94.40%; precision:  44.26%; recall:  72.97%; FB1:  55.10%
                X: precision:  44.26%; recall:  72.97%; FB1:  55.10%  61


...

processed 1125 tokens with 37 phrases; found: 68 phrases; correct: 26.
accuracy:  92.71%; (non-O)
accuracy:  92.71%; precision:  38.24%; recall:  70.27%; FB1:  49.52%
                X: precision:  38.24%; recall:  70.27%; FB1:  49.52%  68


,F1,Precision,Recall
0,49.52,38.24,70.27


<ipython-input-7-05736d5e33a3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['test_ready_prompt'] = [few_shot_prompt+'\n\n'+empty_prompt for empty_prompt in test_df['empty_prompts']]
50it [00:24,  2.03it/s]

processed 1125 tokens with 37 phrases; found: 68 phrases; correct: 23.
accuracy:  90.76%; (non-O)
accuracy:  90.76%; precision:  33.82%; recall:  62.16%; FB1:  43.81%
                X: precision:  33.82%; recall:  62.16%; FB1:  43.81%  68


,F1,Precision,Recall
0,43.81,33.82,62.16


In [12]:
disease_performance_by_prompt_sel_stratified.sort_values('f1',ascending=False)[["few_shot_seed", "dev_set_seed", "f1", "precision", "recall"]]

,few_shot_seed,dev_set_seed,f1,precision,recall
2,31,0,41.46,37.78,45.95
3,9,0,41.46,37.78,45.95
1,4,0,37.04,34.09,40.54
0,42,0,36.59,33.33,40.54


In [13]:
for i in few_shot_seeds:
    print('\n'+str(i))
    print(disease_results_by_prompt_sel_stratified[i,0].test_ready_prompt.values[0])


42
Sentence: As a consequence of blocking I (f), clonidine reduced the slope of the diastolic depolarization and the frequency of pacemaker potentials in sinoatrial node cells from wild-type and alpha2ABC-knockout mice.
Diseases: 

Sentence: Among women who used oral contraceptives, the odds ratio was 2.1 (95 percent confidence interval, 1.5 to 3.0) for those without a prothrombotic mutation and 1.9 (95 percent confidence interval, 0.6 to 5.5) for those with a mutation CONCLUSIONS: The risk of myocardial infarction was increased among women who used second-generation oral contraceptives.
Diseases: myocardial infarction

Sentence: We report the increased amount of motor disability in four patients with idiopathic Parkinson's disease after exposure to the antidepressant fluoxetine.
Diseases: motor disability, idiopathic parkinson's disease

Sentence: Severe and long lasting cholestasis after high-dose co-trimoxazole treatment for Pneumocystis pneumonia in HIV-infected patients--a report

In [33]:
disease_performance_by_prompt_sel, disease_results_by_prompt_sel = test_prompt_selection('davinci',
                                                                                         train_half1, 
                                                                                         train_half2, 
                                                                                         [0,2,3],
                                                                                         [0,1])

/home/jimenezgutierrez.1/biomedical-information-extraction/src/data_utils.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['test_ready_prompt'] = [few_shot_prompt+'\n\n'+empty_prompt for empty_prompt in test_df['empty_prompts']]
50it [00:19,  2.59it/s]
0it [00:00, ?it/s]

processed 1125 tokens with 37 phrases; found: 32 phrases; correct: 18.
accuracy:  95.73%; (non-O)
accuracy:  95.73%; precision:  56.25%; recall:  48.65%; FB1:  52.17%
                X: precision:  56.25%; recall:  48.65%; FB1:  52.17%  32


50it [00:21,  2.35it/s]
0it [00:00, ?it/s]

processed 1470 tokens with 57 phrases; found: 47 phrases; correct: 23.
accuracy:  93.40%; (non-O)
accuracy:  93.40%; precision:  48.94%; recall:  40.35%; FB1:  44.23%
                X: precision:  48.94%; recall:  40.35%; FB1:  44.23%  47


50it [00:23,  2.14it/s]
0it [00:00, ?it/s]

...

processed 1470 tokens with 57 phrases; found: 43 phrases; correct: 20.
accuracy:  93.33%; (non-O)
accuracy:  93.33%; precision:  46.51%; recall:  35.09%; FB1:  40.00%
                X: precision:  46.51%; recall:  35.09%; FB1:  40.00%  43


50it [00:20,  2.45it/s]
0it [00:00, ?it/s]

processed 1125 tokens with 37 phrases; found: 26 phrases; correct: 19.
accuracy:  96.62%; (non-O)
accuracy:  96.62%; precision:  73.08%; recall:  51.35%; FB1:  60.32%
                X: precision:  73.08%; recall:  51.35%; FB1:  60.32%  26


50it [00:19,  2.51it/s]

processed 1470 tokens with 57 phrases; found: 35 phrases; correct: 24.
accuracy:  94.69%; (non-O)
accuracy:  94.69%; precision:  68.57%; recall:  42.11%; FB1:  52.17%
                X: precision:  68.57%; recall:  42.11%; FB1:  52.17%  35


In [266]:
disease_performance_by_prompt_sel.sort_values('f1',ascending=False)[["few_shot_seed", "dev_set_seed", "f1", "precision", "recall"]]

,few_shot_seed,dev_set_seed,f1,precision,recall
2,2,0,65.00,60.47,70.27
6,6,0,60.32,73.08,51.35
4,4,0,53.33,52.63,54.05
0,0,0,52.17,56.25,48.65
7,6,1,52.17,68.57,42.11
1,0,1,44.23,48.94,40.35
3,2,1,43.48,43.10,43.86
5,4,1,40.00,46.51,35.09


In [70]:
disease_performance_by_prompt_sel.groupby('few_shot_seed').agg({'mean','std'}).sort_values(('f1','mean'),ascending=False)

dev_set_seed         f1       precision       recall      
                       std mean   std  mean       std  mean    std  mean
few_shot_seed                                                           
6                     0.71 0.50  5.76 56.25      3.19 70.82   6.54 46.73
2                     0.71 0.50 15.22 54.24     12.28 51.78  18.68 57.06
0                     0.71 0.50  5.62 48.20      5.17 52.59   5.87 44.50
4                     0.71 0.50  9.43 46.67      4.33 49.57  13.41 44.57

In [16]:
# pickle.dump(disease_results_by_prompt_sel, open('prompt_selection_disease_results.no_stratify.p','wb'))
# pickle.dump(disease_results_by_prompt_sel_many_ents, open('prompt_selection_disease_results.more_than_3_ents_only.p','wb'))
pickle.dump(disease_results_by_prompt_sel_stratified, open('prompt_selection_disease_results.stratify.p','wb'))

In [59]:
print(disease_results_by_prompt_sel[(4,1)].test_ready_prompt.values[0])

Sentence: Desferrioxamine withdrawal resulted in a complete recovery of visual function in 1 patient and partial recovery in 3 , and a complete reversal of hearing loss in 3 patients and partial recovery in 3 .
Diseases: hearing loss

Sentence: The block was successful and surgery was conducted as scheduled despite persisting atrial fibrillation .
Diseases: atrial fibrillation

Sentence: An objective causality assessment revealed that the adverse drug event was probably related to the use of ticlopidine .
Diseases: 

Sentence: CONCLUSION : ATT - ALF constituted 5 . 7 % of ALF at our center and had a high mortality rate .
Diseases: alf

Sentence: A policy of unrestricted prescription of appetite suppressants may lead to a high incidence of associated primary pulmonary hypertension .
Diseases: primary pulmonary hypertension

Sentence: METHODS : We present the first case report of a woman with hyperthyroidism treated with propylthiouracil in whom a syndrome of pericarditis , fever , and g

In [40]:
print(disease_results_by_prompt_sel[(2,1)].test_ready_prompt.values[0])

Sentence: We present a 43 - year - old man who developed a coronary aneurysm in the right coronary artery 6 months after receiving a paclitaxel - eluting stent .
Diseases: coronary aneurysm

Sentence: In 44 ( 62 . 8 % ) patients , ATT was prescribed empirically without definitive evidence of tuberculosis .
Diseases: tuberculosis

Sentence: The aim of the present study was to investigate changes in the plasma calcitonin gene - related peptide ( CGRP ) concentration and platelet serotonin ( 5 - hydroxytriptamine , 5 - HT ) content during the immediate headache and the delayed genuine migraine attack provoked by nitroglycerin .
Diseases: headache, migraine

Sentence: Heparan sulphate - associated anionic sites in the glomerular basement membrane were studied in rats 8 months after induction of diabetes by streptozotocin and in age - adn sex - matched control rats , employing the cationic dye cuprolinic blue .
Diseases: diabetes

Sentence: EBFF did not change during PGE1 infusion whereas i

In [39]:
print(disease_results_by_prompt_sel[(0,0)].test_ready_prompt.values[0])

Sentence: In recent years working memory deficits have been reported in users of MDMA ( 3 , 4 - methylenedioxymethamphetamine , ecstasy ) .
Diseases: memory deficits

Sentence: Molecularly , ANF mRNA increased 250 % and SERCA2 mRNA decreased 57 % .
Diseases: 

Sentence: Since nonsteroidal anti - inflammatory agents interfere with this compensatory mechanism and may cause acute renal failure , they should be used with caution in such patients .
Diseases: acute renal failure

Sentence: In conclusion , CNS complications are frequent events during ALL therapy , and require rapid detection and prompt treatment to limit permanent damage .
Diseases: all

Sentence: The relative amounts of alphaENaC , betaENaC and gammaENaC mRNAs were determined in kidneys from these rats by real - time quantitative TaqMan PCR , and the amounts of proteins by Western blot .
Diseases: 

Sentence: The site of common side effects of sumatriptan .
Diseases:


In [58]:
print(disease_results_by_prompt_sel[(6,0)].test_ready_prompt.values[0])

Sentence: Furthermore , the effects are mediated through dopamine rather than norepinephrine and do not require the carotid sinus baroreceptors .
Diseases: 

Sentence: Amiloride reduced the drinking and urine volume of rats in an acute ( 6 or 12 h ) and a subacute ( 3 days ) experiment .
Diseases: 

Sentence: An electroencephalogram showed continuous , generalized irregular slowing with admixed periodic triphasic waves indicating symptomatic encephalopathy .
Diseases: encephalopathy

Sentence: Whatever was the dose , the central administration of U - II had no effect on body temperature , nociception , apomorphine - induced penile erection and climbing behavior , and stress - induced plasma corticosterone level .
Diseases: penile erection

Sentence: Most of these patients had more than two metastatic sites , with lung metastasis predominant .
Diseases: 

Sentence: The site of common side effects of sumatriptan .
Diseases:


In [250]:
def get_word_level_results(df_dict):
    result_by_seeds = []

    for s1,s2 in df_dict.keys(): 
        result_df = df_dict[(s1,s2)]

        df = create_bio_preds(result_df, "predictions")
        b_true = [s.replace('I','B') for s in df.ner_seq]
        b_pred = [s.replace('I','B') for s in df.bio_preds]

        f1, precision, recall = conlleval_eval(b_true,b_pred)

        result_by_seeds.append((s1, s2, f1, precision, recall))
        
    return pd.DataFrame(result_by_seeds).sort_values(2)

def get_entity_level_results(df_dict):
    result_by_seeds = []

    for s1,s2 in df_dict.keys(): 
        result_df = df_dict[(s1,s2)]

        df = create_bio_preds(result_df, "predictions")
        b_true = df.ner_seq
        b_pred = df.bio_preds

        f1, precision, recall = conlleval_eval(b_true,b_pred)

        result_by_seeds.append((s1, s2, f1, precision, recall))
        
    return pd.DataFrame(result_by_seeds).sort_values(2,ascending=False)

In [251]:
get_word_level_results(disease_results_by_prompt_sel_many_ents)

processed 1125 tokens with 55 phrases; found: 178 phrases; correct: 47.
accuracy:  87.64%; (non-O)
accuracy:  87.64%; precision:  26.40%; recall:  85.45%; FB1:  40.34%
                X: precision:  26.40%; recall:  85.45%; FB1:  40.34%  178
processed 1125 tokens with 55 phrases; found: 203 phrases; correct: 46.
accuracy:  85.24%; (non-O)
accuracy:  85.24%; precision:  22.66%; recall:  83.64%; FB1:  35.66%
                X: precision:  22.66%; recall:  83.64%; FB1:  35.66%  203


,0,1,2,3,4
1,2,0,35.66,22.66,83.64
0,0,0,40.34,26.40,85.45


In [252]:
get_word_level_results(disease_results_by_prompt_sel_stratified)

processed 1125 tokens with 55 phrases; found: 175 phrases; correct: 50.
accuracy:  88.44%; (non-O)
accuracy:  88.44%; precision:  28.57%; recall:  90.91%; FB1:  43.48%
                X: precision:  28.57%; recall:  90.91%; FB1:  43.48%  175
processed 1125 tokens with 55 phrases; found: 95 phrases; correct: 46.
accuracy:  94.84%; (non-O)
accuracy:  94.84%; precision:  48.42%; recall:  83.64%; FB1:  61.33%
                X: precision:  48.42%; recall:  83.64%; FB1:  61.33%  95
processed 1125 tokens with 55 phrases; found: 107 phrases; correct: 42.
accuracy:  93.07%; (non-O)
accuracy:  93.07%; precision:  39.25%; recall:  76.36%; FB1:  51.85%
                X: precision:  39.25%; recall:  76.36%; FB1:  51.85%  107
processed 1125 tokens with 55 phrases; found: 130 phrases; correct: 42.
accuracy:  91.02%; (non-O)
accuracy:  91.02%; precision:  32.31%; recall:  76.36%; FB1:  45.41%
                X: precision:  32.31%; recall:  76.36%; FB1:  45.41%  130


,0,1,2,3,4
0,42,0,43.48,28.57,90.91
3,9,0,45.41,32.31,76.36
2,31,0,51.85,39.25,76.36
1,4,0,61.33,48.42,83.64


In [265]:
get_entity_level_results(disease_results_by_prompt_sel_stratified)

processed 1125 tokens with 37 phrases; found: 93 phrases; correct: 26.
accuracy:  87.91%; (non-O)
accuracy:  87.91%; precision:  27.96%; recall:  70.27%; FB1:  40.00%
                X: precision:  27.96%; recall:  70.27%; FB1:  40.00%  93
processed 1125 tokens with 37 phrases; found: 60 phrases; correct: 27.
accuracy:  94.49%; (non-O)
accuracy:  94.49%; precision:  45.00%; recall:  72.97%; FB1:  55.67%
                X: precision:  45.00%; recall:  72.97%; FB1:  55.67%  60
processed 1125 tokens with 37 phrases; found: 68 phrases; correct: 26.
accuracy:  92.71%; (non-O)
accuracy:  92.71%; precision:  38.24%; recall:  70.27%; FB1:  49.52%
                X: precision:  38.24%; recall:  70.27%; FB1:  49.52%  68
processed 1125 tokens with 37 phrases; found: 68 phrases; correct: 23.
accuracy:  90.58%; (non-O)
accuracy:  90.58%; precision:  33.82%; recall:  62.16%; FB1:  43.81%
                X: precision:  33.82%; recall:  62.16%; FB1:  43.81%  68


,0,1,2,3,4
1,4,0,55.67,45.00,72.97
2,31,0,49.52,38.24,70.27
3,9,0,43.81,33.82,62.16
0,42,0,40.00,27.96,70.27


In [71]:
result_by_seeds.groupby(0).agg({'mean','std'}).sort_values((2,'mean'),ascending=False)

1          2          3           4      
   std mean   std  mean  std  mean   std  mean
0                                             
2 0.71 0.50  4.65 60.43 1.31 61.14  7.84 59.91
6 0.71 0.50  3.71 54.20 1.80 74.49  3.99 42.63
4 0.71 0.50 11.33 49.93 3.48 57.15 15.83 45.17
0 0.71 0.50  1.89 49.69 1.73 56.92  1.93 44.09

In [103]:
result_by_seeds.sort_values(2,ascending=False)

,0,1,2,3,4
2,2,0,63.72,62.07,65.45
4,4,0,57.94,59.62,56.36
3,2,1,57.14,60.22,54.37
6,6,0,56.82,75.76,45.45
7,6,1,51.57,73.21,39.81
0,0,0,51.02,58.14,45.45
1,0,1,48.35,55.70,42.72
5,4,1,41.92,54.69,33.98
